In [1]:
from google.colab import drive
import os

# set path to project folder
gdrive_path='/content/gdrive/MyDrive/1-university/masters/2-semester/in2390_adl4cv/nerf_segmentation/' # Luca's Path
# gdrive_path='/content/gdrive/MyDrive/...' # Luis' Path

# mount Google Drive
drive.mount('/content/gdrive', force_remount=True)

# navigate to Google Drive folder
os.chdir(gdrive_path)

# check that we are in the right folder
print(sorted(os.listdir()))

Mounted at /content/gdrive
['.git', '.gitignore', '.ipynb_checkpoints', 'PeRFception-ScanNet', 'README.md', 'data', 'data_loader.ipynb', 'untitled']


Execute the following code from your machine's terminal:
```
git clone https://huggingface.co/datasets/YWjimmy/PeRFception-ScanNet
```

Then compress the repository into a .zip-file and place it inside the `/data/` folder of this repository. This step is necessary to allow the execution of this notebook inside Google Colab. In our next step, we will unzip the file inside of this Colab session:


In [2]:
!unzip "/content/gdrive/MyDrive/1-university/masters/2-semester/in2390_adl4cv/nerf_segmentation/data/PeRFception-ScanNet.zip"

Archive:  /content/gdrive/MyDrive/1-university/masters/2-semester/in2390_adl4cv/nerf_segmentation/data/PeRFception-ScanNet.zip
replace PeRFception-ScanNet/README.md? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace PeRFception-ScanNet/.gitattributes? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace PeRFception-ScanNet/plenoxel_scannet_scene0067_00/thick.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace PeRFception-ScanNet/plenoxel_scannet_scene0067_00/last.ckpt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!python -m pip install torch torchvision torchaudio plyfile MinkowskiEngine

In [18]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

MinkowskiEngine is not compatible with Python3.10. You have to go to `/usr/local/lib/python3.10/dist-packages/MinkowskiEngine/` and adjust some packages to use updated imports for `Sequence` that was previously part of `collections` and is now part of `collections.abc` as mentioned here: https://github.com/NVIDIA/MinkowskiEngine/issues/526

In [17]:
from torch.utils.data import Dataset
import numpy as np
import logging
import MinkowskiEngine as ME
from plyfile import PlyData

def load_ply(filename, load_label=False, load_instance=False):
    plydata = PlyData.read(filename)
    data = plydata.elements[0].data
    coords = np.array([data["x"], data["y"], data["z"]], dtype=np.float32).T
    feats = np.array([data["red"], data["green"], data["blue"]], dtype=np.float32).T
    return_args = [coords, feats]
    if load_label:
        labels = np.array(data["label"], dtype=np.int32)
        return_args.append(labels)
    if load_instance:
        instances = np.array(data["instance"], dtype=np.int32)
    else:
        instances = np.ones(coords.shape[0], dtype=np.int32)
    return_args.append(instances)
    return tuple(return_args)

class PlenoxelScannetDataset(Dataset):
    DATA_PATH_FILE = {
        "train": "scannetv2_train.txt",
        "val": "scannetv2_val.txt",
        "test": "scannetv2_test.txt",
    }

    def __init__(
        self,
        phase: str,
        data_root: str = "datasets/scannet",
        downsample_voxel_size=None,  # in meter
        voxel_size=0.02,
        train_transformations=[
            "ChromaticTranslation",
            "ChromaticJitter",
            "CoordinateDropout",
            "RandomHorizontalFlip",
            "RandomAffine",
            "RandomTranslation",
            "NormalizeColor",
        ],
        eval_transformations=[
            "NormalizeColor",
        ],
        ignore_label=-100,
        features=["colors"],  # ["colors", "xyzs"]
    ):
        Dataset.__init__(self)
        self.ignore_label = ignore_label
        self.data_root = data_root
        self.phase = phase
        transformations = (
            train_transformations if phase == "train" else eval_transformations
        )

        #self.transformations = transforms.Compose(
        #    [transforms.__dict__[t]() for t in transformations]
        #)

        self.pc_files = []
        with open(os.path.join(self.data_root, self.DATA_PATH_FILE[phase]), "r") as f:
            self.pc_files.extend([l.rstrip("\n") for l in f.readlines()])

        if downsample_voxel_size is None:
            downsample_voxel_size = voxel_size / 2
        self.downsample_voxel_size = downsample_voxel_size
        self.voxel_size = voxel_size

        # map labels not evaluated to ignore_label
        label_map, n_used = dict(), 0
        for l in range(self.NUM_LABELS):
            if l in self.IGNORE_LABELS:
                label_map[l] = ignore_label
            else:
                label_map[l] = n_used
                n_used += 1
        label_map[ignore_label] = ignore_label
        self.label_map = label_map
        self.features = features

    def __len__(self):
        return len(self.pc_files)

    def __getitem__(self, i: int):
        xyzs, colors, labels, instances = load_ply(
            os.path.join(self.data_root, self.pc_files[i]),
            load_label=True,
            load_instance=True,
        )

        if self.downsample_voxel_size > 0:
            pre_len = len(xyzs)
            _, colors, labels, row_inds = ME.utils.sparse_quantize(
                np.ascontiguousarray(xyzs),
                colors,
                labels=labels,
                quantization_size=self.downsample_voxel_size,
                return_index=True,
                ignore_label=self.ignore_label,
            )
            # Maintain the continuous coordinates
            xyzs = xyzs[row_inds] / self.voxel_size
            instances = instances[row_inds]
            logging.debug(f"Downsampled point cloud index {i} from {pre_len} to {xyzs}")
        else:
            xyzs /= self.voxel_size

        xyzs, colors, labels = self.transformations(xyzs, colors, labels)
        # instances_info = self.get_instance_info(xyzs, instances)
        # centers = instances_info["center"]
        # instance_ids = instances_info["ids"]
        if self.IGNORE_LABELS is not None:
            labels = np.array([self.label_map[x] for x in labels], dtype=np.int)

        features = []
        for f in self.features:
            features.append(eval(f))
        return {
            "coordinates": xyzs.astype(np.float32),
            "features": np.concatenate(features, axis=1).astype(np.float32),
            "labels": labels,
            "colors": colors,
            # "instance_centers": centers,
            # "instance_ids": instance_ids,
            "dataset": "scannet",
        }



/usr/local/lib/python3.10/dist-packages/MinkowskiEngine/__init__.py:221: UserWarning: The MinkowskiEngine was compiled with CPU_ONLY flag. If you want to compile with CUDA support, make sure `torch.cuda.is_available()` is True when you install MinkowskiEngine.
  warnings.warn(


In [19]:
dataset = PlenoxelScannetDataset("train")

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/scannet/scannetv2_train.txt'